# Yabble's AI Test

Hello! Given the rise of code generation LLMs traditional coding tests are completely useless these days. So we've come up with something that should be hard for an AI to code without some sort of direction from a human, so this means this test is really about giving you a problem and seeing how you 

- Solve a problem with LLMs, and
- Communicate this solution.

We're mainly see want to see a proof-of-concept so doesn't need to be an end-to-end working solution. Just a sketch so we can see your thinking behind your solution.

Fork this repo and send us a link when you are done to alex@yabble.com

## The Problem

As part of this repo we have supplied a dataset `walmart.csv` there are 3 columns of open text responses. The task is to perform, what the market research community call, "theme counting" or "thematic analysis: on one or all of these columns. A brief definition of this task might be:

> [Thematic analysis](https://fastercapital.com/content/Thematic-Analysis--Uncovering-Patterns--Thematic-Analysis-in-Qualitative-Research.html) is a qualitative research technique that involves systematically identifying, analyzing, and reporting patterns (themes) within textual, visual, or audio data. It is widely used in marketing research to explore consumer opinions, sentiments, and experiences. Researchers begin by immersing themselves in the data, identifying recurring patterns, and organizing them into meaningful themes.

### Example

So a basic example would be if you had 3 responses from the dataset column "What do you think Target does better than Walmart?":
1. "Their overall approach to customers and the benefits of supporting the community and it's employees"
2. "Target has a better variety"
3. "Employees are more helpful and the stores are much cleaner and better organized"
4. "They have more in style things and the employees are nice"

So you might tags these with "themes" as follows:
1. Customer support; Supports Community; Supports Employees
2. Variety
3. Customer support; Clean stores; Well Organised
4. Variety; Customer support

The idea with themes is to group responses to make further porcess easier. You need to find a balance between having general themes while at the same time capturing enough neuonce for the themes to be useful. 

Bonus marks for adding sentiment to your themes so the same theme can ether be positive, negative or neutral i.e. Customer Service (positive) or Customer Service (Negative) etc

### Please...
- Document your code and your thinking. 
- Doesn't have to be perfect, show the experiments you did to get to where you ended up. 
- Link to external sources of inspiration.
- Write a "Summary" at the begining which outlines your approch(es)
- Write a "Conclusion" section at the end.

Good luck!

## The data

The data is a typical CSV we migth get from a client. Column labels are the survey question text.

In [2]:
import csv

with open("walmart.csv") as fp:
    reader = csv.DictReader(fp)
    data = [row for row in reader]

data[:3]

[{'Session ID': '94f2d4c3-b513-411c-b505-a11290b3f0f5',
  'Date started': '10/02/2022 01:03',
  'Imagine you are writing a complaint letter to the head of Walmart...in detail, tell us the things you would complain about and why they matter?': "The customer service that your company gives is horrible and tacky. Your company has no morality or empathy for it's customers \nAll you care about is profits over customer.",
  'What do you think Target does better than Walmart?': "Their overall approach to customers and the benefits of supporting the community and it's employees.",
  'What do you LOVE most about Walmart?': "There's nothing I LOVE about Walmart.",
  'Gender': 'Male',
  'Age group': '45-54',
  'State': 'Texas',
  'Who does it better? Target or Walmart?': 'Target'},
 {'Session ID': '1797c6f2-c501-44b7-b549-a33c29224edc',
  'Date started': '10/02/2022 01:03',
  'Imagine you are writing a complaint letter to the head of Walmart...in detail, tell us the things you would complain abou

In [3]:
# Your code here...